In [31]:
%load_ext autoreload
%autoreload 2

import os
import logging
import json

from tradingdb.utils import *
from tradingdb.ORM.utils import *
from sqlalchemy import exc
import sqlite3


logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
handler.setLevel(logging.NOTSET)
format = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
handler.setFormatter(format)

logger.addHandler(handler)

from datetime import datetime, timedelta


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Params

In [3]:
intervals_to_add=['1d']
DBNAME=os.path.abspath("../data/master.db")
deltadays=14

In [2]:
time_end=datetime.today().replace(hour=4, minute=30, microsecond=0)
# covid crisis: start new era from ftse slump (23 march)
time_start = time_end - timedelta(days=deltadays)
time_start=time_start.replace(hour=8, minute=0, microsecond=0)
print(time_start,time_end)

2023-07-24 08:00:11 2023-08-07 04:30:11


In [41]:

def retrieve_new_period_for_sid(session,sid,time_start,time_end,ival):
    # get the yf symbol
    with session.begin() as mysession:
        q=mysession.query(stock.Stock)
        ret=q.filter(stock.Stock.id==sid).one()

        pid=mysession.query(stock.Period.id).filter(stock.Period.name==ival).one()[0]

        print(ret.yf_symbol,ival)
        dfd=download_data_from_yfsymbols(ret.yf_symbol,time_start,time_end,interval=ival)
        for index, row in dfd[ret.yf_symbol].iterrows():        
            try:
                subsession=session()
                qq=subsession.query(stock.Price.id)
                qq=qq.query()
                p=stock.Price(
                            stock_id=sid,
                            period_id=pid,
                            date=index.to_pydatetime(),  
                            open=row['Open'],
                            low=row['Low'],
                            high=row['High'],
                            close=row['Close'],
                            adjusted_close=row['Adj Close'],
                            volume=row['Volume'])
                subsession.add(p)
                subsession.flush()
            except sqlite3.IntegrityError as e:
                subsession.rollback()  # Rollback the changes to avoid leaving the session in an invalid state
                print(ret.yf_symbol,ival,index.to_pydatetime(),"not inserted")
                #print("IntegrityError: {}".format(e))
                #break
                subsession.close()
            except Exception as e:
                subsession.rollback()  # Rollback the changes for other unexpected exceptions
                #print("Unexpected error: {}".format(e))
                print(ret.yf_symbol,ival,index.to_pydatetime(),"not inserted")

                #break
                subsession.close()



session=get_new_session(DBNAME)
with session.begin() as mysession:
    ret=mysession.query(stock.Stock.id).all()
sids=[r[0] for r in ret]
for ival in intervals_to_add:
    for sid in sids:
        with session.begin() as mysession:
            q=mysession.query(stock.Stock).join(stock.Price).join(stock.Period)
            ret=q.filter(stock.Stock.id==sid).filter(stock.Period==ival).all()
        if len(ret)==0:
            print("Retrieving interval for ",sid)

            retrieve_new_period_for_sid(session,sid,time_start,time_end,ival)
            #break
    #break


89 0
AAF.L 1d
[*********************100%***********************]  1 of 1 completed
AAF.L 1d 2023-07-24 00:00:00 not inserted
AAF.L 1d 2023-07-25 00:00:00 not inserted
AAF.L 1d 2023-07-26 00:00:00 not inserted
AAF.L 1d 2023-07-27 00:00:00 not inserted
AAF.L 1d 2023-07-28 00:00:00 not inserted
AAF.L 1d 2023-08-01 00:00:00 not inserted
AAF.L 1d 2023-08-02 00:00:00 not inserted
AAF.L 1d 2023-08-03 00:00:00 not inserted


sqlalchemy.pool.impl.NullPool - ERROR - Exception during reset or similar
Traceback (most recent call last):
  File "/Users/davidebranduardi/Projects/Finance/trading-toolbox/tradingdb/tradingdb/condavenv/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 757, in _finalize_fairy
    fairy._reset(pool, transaction_was_reset)
  File "/Users/davidebranduardi/Projects/Finance/trading-toolbox/tradingdb/tradingdb/condavenv/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 1016, in _reset
    pool._dialect.do_rollback(self)
  File "/Users/davidebranduardi/Projects/Finance/trading-toolbox/tradingdb/tradingdb/condavenv/lib/python3.9/site-packages/sqlalchemy/engine/default.py", line 683, in do_rollback
    dbapi_connection.rollback()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 4685217280 and this is thread id 123145552912384.
sqlalchemy.pool.impl.NullPool - ERROR - Exception during reset or

AAF.L 1d 2023-08-04 00:00:00 not inserted
36 0
AAL.L 1d
[*********************100%***********************]  1 of 1 completed
AAL.L 1d 2023-07-24 00:00:00 not inserted
AAL.L 1d 2023-07-25 00:00:00 not inserted
AAL.L 1d 2023-07-26 00:00:00 not inserted
AAL.L 1d 2023-07-27 00:00:00 not inserted
AAL.L 1d 2023-07-28 00:00:00 not inserted
AAL.L 1d 2023-07-31 00:00:00 not inserted
AAL.L 1d 2023-08-01 00:00:00 not inserted
AAL.L 1d 2023-08-02 00:00:00 not inserted
AAL.L 1d 2023-08-03 00:00:00 not inserted
AAL.L 1d 2023-08-04 00:00:00 not inserted
108 0
AAPL 1d
[*********************100%***********************]  1 of 1 completed
AAPL 1d 2023-07-24 00:00:00 not inserted
AAPL 1d 2023-07-25 00:00:00 not inserted
AAPL 1d 2023-07-26 00:00:00 not inserted
AAPL 1d 2023-07-27 00:00:00 not inserted
AAPL 1d 2023-07-28 00:00:00 not inserted
AAPL 1d 2023-08-01 00:00:00 not inserted
AAPL 1d 2023-08-02 00:00:00 not inserted
AAPL 1d 2023-08-03 00:00:00 not inserted
AAPL 1d 2023-08-04 00:00:00 not inserted
25 

KeyboardInterrupt: 

In [22]:
with session.begin() as mysession:
    mysession.rollback()